In [1]:
import os
import numpy as np
import math
import pandas as pd
import cv2
import PIL.Image
from IPython.display import Image
import random

In [2]:
os.chdir("../")
os.getcwd()

'/home/honza/Documents/UniHack/fast-form'

## get dataset

In [3]:
train_db = pd.read_csv("training_data/emnist-balanced-train.csv")
test_db  = pd.read_csv("training_data/emnist-balanced-test.csv")

In [11]:
def convert_row_to_img(img):
    img = img.astype('uint8')
    img = img.reshape((28,28))
    img = img.transpose()
    # img = cv2.bitwise_not(img)
    return img


def convert_img_to_row(img):
    # img = cv2.bitwise_not(img)
    img = img.transpose()
    img = img.ravel()
    img = img.astype('int64') 
    return img


def resize(img, dimension):
    w, h = img.shape
    maxi = max([w, h])

    r = dimension / maxi
    new_w = math.floor(w * r)
    new_h = math.floor(h * r)

    res = cv2.resize(img, dsize=(new_h, new_w), interpolation=cv2.INTER_CUBIC)
    if max(res.shape) != dimension:
        raise ValueError("not good size of resized shape, fix!")

    bottom = math.ceil((dimension - new_w) / 2)
    top = math.floor((dimension - new_w) / 2)
    right = math.ceil((dimension - new_h) / 2)
    left = math.floor((dimension - new_h) / 2)
    res = cv2.copyMakeBorder(res, top, bottom, left, right, cv2.BORDER_CONSTANT, value=255)

    return res


def add_radnom_border(img, target_size = 28, max_downsize = 8):
    # downsize 
    downsize = 1 + round(random.random() * (max_downsize-1))
    new_size = target_size - downsize
    downsize_img = resize(img, new_size)

    # pick random border
    left = round(random.random () * downsize)
    right = downsize - left
    top = round(random.random () * downsize)
    bottom = downsize - top

    # add border
    res = cv2.copyMakeBorder(downsize_img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=0)
    return res

    
def transform_img(data):
    # get label
    label = data.values[0]
    
    # add border
    img = data.values[1:]
    img = convert_row_to_img(img)
    new_img = add_radnom_border(img)
    
    # corvert to row
    new_row = convert_img_to_row(new_img)
    
    # add label
    new_row = np.concatenate([[label],new_row])
    
    return new_row


def transform_dataframe(df):
    columns = df.columns
#     new_chars = df[:1000].apply(transform_img, axis = 1)
    new_chars = df.apply(transform_img, axis = 1)
    new_df = pd.DataFrame(list(new_chars), columns=columns)
    return new_df


def show(img):
    display(PIL.Image.fromarray(img))

In [12]:
# test
img_row = train_db.iloc[1].values
img_row = img_row[1:]

img = convert_row_to_img(img_row)
img.shape
for i in range(5):
    new_img = add_radnom_border(img)
    show(new_img)

## enrich training set

In [13]:
new_train_db = transform_dataframe(train_db)
new_train_db = train_db.append(new_train_db)
print(f"Original size: {train_db.shape}")
print(f"New size:      {new_train_db.shape}")

Original size: (112799, 785)
New size:      (225598, 785)


## enrich testing set

In [14]:
new_test_db = transform_dataframe(test_db)
new_test_db = test_db.append(new_test_db)
print(f"Original size: {test_db.shape}")
print(f"New size:      {new_test_db.shape}")

Original size: (18799, 785)
New size:      (37598, 785)


## save

In [15]:
new_train_db.to_csv('training_data/emnist-balanced-enriched-train.csv', index=False)
new_test_db.to_csv('training_data/emnist-balanced-enriched-test.csv', index=False)

In [16]:
# train_db = pd.read_csv("training_data/emnist-balanced-enriched-train.csv")
test_db  = pd.read_csv("training_data/emnist-balanced-enriched-test.csv")

In [17]:
test_db.head()

,41,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,0.523,0.524,0.525,0.526,0.527,0.528,0.529,0.530,0.531,0.532
0,39,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,44,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,33,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
